<a href="https://colab.research.google.com/github/CristianMorasso/statistical_project/blob/master/Copia_di_statiscal_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Statistical Learning Project: ALFA Dataset Anaysis


In [ ]:
!pip install icecream sktime scikit-learn polars numpy
!git clone https://github.com/CristianMorasso/statistical_project.git


# # Connect Drive to get the Dataset
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)
# dataset_dir= '/content/drive/MyDrive/alfa_dataset'


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.1/130.1 kB 9.0 MB/s eta 0:00:00
Cloning into 'statistical_project'...
remote: Enumerating objects: 70, done.
remote: Total 70 (delta 0), reused 0 (delta 0), pack-reused 70
Receiving objects: 100% (70/70), 90.30 MiB | 19.55 MiB/s, done.
Updating files: 100% (62/62), done.


In [ ]:
dataset_dir = "statistical_project/data"

In [ ]:
import pandas  as pd
import os

dfs = [] # dataframe list
class_dict = {}
for i, cl  in  enumerate(os.listdir(dataset_dir)):
  path1 = os.path.join(dataset_dir, cl)
  if not os.path.isdir(path1) or cl == "features":
    print(f"-{cl} is not a dir")
    continue
  class_dict[i] = cl
  print(f"{cl}")
  for f in  os.listdir(os.path.join(dataset_dir, cl)):
    pathf = os.path.join(path1, f)
    df = pd.read_csv(pathf)

    try:
      df.drop("field.data", axis=1)
    except:
      continue

    df.insert(0, "Y", [ i for _ in range(len(df))], True)
    dfs.append(df)




ailerons
elevator
rudder
-README.md is not a dir
no_failure
-__init__.py is not a dir
-features is not a dir
engine


In [ ]:
# Select the k Best features
from sklearn import feature_selection
print(f"Number of columns:{len(dfs[0].columns)}")
# SelectKBest

Number of columns:28


In [ ]:
classes = set()
for df in dfs:
  classes.add(df["y"][0])
  print(df["y"][0],df.shape) #Le TS hanno lunghezza diversa valutare come risolvere
print("classes: ", classes)

classes:  {'no_failure', 'engine', 'elevator', 'ailerons', 'rudder'}


## Feature selection

In [ ]:

from sklearn.feature_selection import SelectKBest
X_new = SelectKBest(k=20).fit(X, y)

NameError: name 'X' is not defined